In [2]:
import pandas as pd

data = pd.read_csv('Updated_Filtered_Dataset.csv')
data.head()

,id,Disruption event,Port Affected,Date,CombinedText,headline,description,severity,main_risk,country,state,link,GloVe_Vectors,Cleaned_Headline,Cleaned_Description,Final Classification,Disruption Event,Date Item,Affected Port
0,66ecf6adb4851f9c3cbdc0ed,Stricter ship insurance requirements aimed at ...,NaN,"June 2nd, 2023 (Circular date)",Stricter ship insurance requirements aimed at ...,Stricter UAE Ship Insurance Rules Look to Cut ...,"LONDON, June 7 (Reuters) Tougher requirements ...",NaN,NaN,NaN,NaN,https://gcaptain.com/stricter-uae-ship-insuran...,[ 5.36050014e-02 1.38071433e-01 1.04028536e-...,Stricter UAE Ship Insurance Rules Look Cut Oil...,"LONDON, June 7 (Reuters) Tougher requirement s...",Environmental Impact and Pollution,UAE introduces stricter ship insurance rules f...,"June 2, 2023",UAE ports
1,66ecf6adb4851f9c3cbdc0ee,"Two major international grain traders, Cargill...",NaN,"May (implied, as the new wheat export season s...","Two major international grain traders, Cargill...",Global Food Supply Risks Rise as Key Traders L...,By ine Quinn and Megan Durisin (Bloomberg) Rus...,NaN,NaN,NaN,NaN,https://gcaptain.com/global-food-supply-risks-...,[ 1.08986357e-02 1.67753562e-01 -5.54059111e-...,Global Food Supply Risks Rise Key Traders Leav...,ine Quinn Megan Durisin (Bloomberg) Russia gri...,Unstable Regulatory and Political Environment,"Major international grain traders, including C...","Not explicitly stated, but the article mention...","Multiple ports in Russia, primarily those hand..."
2,66ecf6adb4851f9c3cbdc0ef,A strike by port workers on Canada's west coast.,Vancouver,Not mentioned.,A strike by port workers on Canada's west coas...,Canada s West Coast Port Strike Seen Putting C...,By Curtis Heinzl (Bloomberg) A on Canada s wes...,NaN,NaN,NaN,NaN,https://gcaptain.com/canadas-west-coast-port-s...,[ 2.56826859e-02 9.78766680e-02 5.26836626e-...,Canada West Coast Port Strike Seen Putting Cou...,Curtis Heinzl (Bloomberg) Canada west coast st...,Port or Important Route Congestion,Labor strike by ILWU at Canada's West Coast po...,5th day of strike,Vancouver
3,66ecf6adb4851f9c3cbdc0f0,Fire on a scrap metal barge towed northbound i...,NaN,"May 23, 2022",Fire on a scrap metal barge towed northbound i...,NTSB Warns of Potential Fire Risks for Vessels...,The National Transportation Safety Board (NTSB...,NaN,NaN,NaN,NaN,https://gcaptain.com/ntsb-warns-of-potential-f...,[-5.26712500e-02 -2.65691400e-01 -2.34190241e-...,NTSB Warns Potential Fire Risks Vessels Carryi...,National Transportation Safety Board (NTSB) is...,Vessel Accidents,Scrap metal cargo fire on a barge in Delaware ...,"May 23, 2022","Newark, New Jersey (shoreside pile) & Delaware..."
4,66ecf6adb4851f9c3cbdc0f1,A tanker arrived to pump over a million barrel...,Hodiedah,"July 16, 2023",A tanker arrived to pump over a million barrel...,Tanker Arrives in Yemen to Start Risky Oil Pum...,"By Mohammed Hatem Jul 16, 2023 (Bloomberg) A a...",NaN,NaN,NaN,NaN,https://gcaptain.com/tanker-arrives-in-yemen-t...,[-1.98929176e-01 1.93371013e-01 2.48778204e-...,Tanker Arrives Yemen Start Risky Oil Pumping O...,"Mohammed Hatem Jul 16, 2023 (Bloomberg) arrive...",Environmental Impact and Pollution,A tanker arrived in Yemen to begin a risky ope...,"July 16, 2023",Hodiedah


In [5]:
data['Final Classification'].value_counts()

Final Classification
Vessel Accidents                                 731
Port or Important Route Congestion               626
Others                                           595
Maritime Piracy/Terrorism Risk                   567
Unstable Regulatory and Political Environment    417
Environmental Impact and Pollution               303
Not maritime-related                             258
Port Criminal Activities                         226
Inland Transportation Risks                      209
Cargo or Ship Detainment                         182
Natural Extreme Events and Extreme Weather       139
Vessel Delay                                     138
Cargo Damage and Loss                             32
Name: count, dtype: int64

In [ ]:
from tqdm import tqdm
import time
import google.generativeai as genai
import os

api_key = os.getenv("GENAI_API_KEY")

genai.configure(api_key=api_key)

few_shot_prompt = """

You are a maritime severity assessor. 
Based on the headline, description, risk type, 
and affected port of each maritime incident, 
assign a severity level (1-5) by considering the following criteria:

- Human Impact: The extent of injuries or fatalities.
- Environmental Impact: Spills, pollution, or ecosystem damage.
- Operational Disruption: Delays, rerouting, or other operational impacts.
- Potential for Escalation: Risk of worsening, like fire or further environmental harm.

Severity levels:
- **1 (Low Severity)**: Minor impact, no injuries, minor cleanup.
- **2 (Moderate Severity)**: Noticeable impact, non-critical injuries, moderate cleanup.
- **3 (High Severity)**: Substantial impact, critical injuries, or multi-hour disruptions.
- **4 (Critical Severity)**: Severe incident, major injuries, significant cleanup.
- **5 (Catastrophic Severity)**: Extreme impact, fatalities, severe long-term consequences.

Respond with only the severity level as a single number (1, 2, 3, 4, or 5).

New Event:
"""

def predict_severity(headline, description, risk_type, affected_port):
    input_prompt = (
        f"{few_shot_prompt}"
        f"**Headline**: {headline}\n"
        f"**Description**: {description}\n"
        f"**Risk Type**: {risk_type}\n"
        f"**Affected Port**: {affected_port}\n"
        "Assessment: Severity Level:"
    )
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(input_prompt)
    severity_level = response.candidates[0].content.parts[0].text.strip()
    time.sleep(5)
    return severity_level
    

batch_size = 10
output_file_path = 'maritime_severity_predictions.csv'

if os.path.exists(output_file_path):
    final_data = pd.read_csv(output_file_path)
    completed_batches = len(final_data) // batch_size
    print(f"Resuming from batch {completed_batches}")
else:
    final_data = pd.DataFrame()
    completed_batches = 0

results = []
for i in tqdm(range(completed_batches * batch_size, len(data), batch_size)):
    batch = data.iloc[i:i + batch_size].copy()
    batch['predicted_severity'] = batch.apply(
        lambda row: predict_severity(row['Cleaned_Headline'], row['Cleaned_Description'], row['Final Classification'], row['Affected Port']), axis=1
    )
    results.append(batch)

    final_data = pd.concat([final_data, batch], ignore_index=True)
    final_data.to_csv(output_file_path, index=False)

print(f"Final predictions saved to {output_file_path}")

In [3]:
df = pd.read_csv("maritime_severity_predictions.csv")
df['predicted_severity'].value_counts()

predicted_severity
2    1483
3    1176
1     836
4     796
5     121
0      11
Name: count, dtype: int64

In [ ]:
severity_mapping = {
    "1.0": "Low",
    "2.0": "Moderate",
    "3.0": "High",
    "4.0": "Critical",
    "5.0": "Catastrophic"
}

df['predicted_severity'] = df['predicted_severity'].astype(str)
df['predicted_severity_label'] = df['predicted_severity'].map(severity_mapping)
df.to_csv("final_data_2.csv", index=False)